In [16]:
#search for change in dice loss for single sample
import pandas as pd
import numpy as np
import os
from monai.metrics import compute_dice
import glob
import torch


In [7]:
dvolumeT = pd.read_csv('/workspaces/data/MegaGen/logs/SCORE/CSVS/id_test_volume.csv')
dvolTS = dvolumeT[dvolumeT['volume'] <3000]

In [13]:
dvolTS['id'] = dvolTS['id'].apply(lambda x: f'{int(x):04d}')

/tmp/ipykernel_17996/802318403.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dvolTS['id'] = dvolTS['id'].apply(lambda x: f'{int(x):04d}')


In [ ]:
root = '/workspaces/data/brain_meningioma'
dScore = pd.DataFrame() # columns=['maskTfile', 'dscore'])
for id in dvolTS['id']:
    maskTfile = os.path.join('npy', f'BraTS-MEN-RT-{id}-1_slice_*_mask.npy')
    # probTfile = os.path.join('pt', f'BraTS-MEN-RT-{id}-1_slice_*_mask.pt')
    maskFiles = glob.glob(os.path.join(root, 'slice/s_test', maskTfile))
    for maskFile in maskFiles:
        probFile0 = os.path.relpath(maskFile, os.path.join(root, 'slice/test')).replace('npy', 'pt')
        probFile0 = os.path.join(root, 'oProb/unet1s/s_test', probFile0)
        y_pred0 = torch.load(probFile0,weights_only=False).unsqueeze(0).unsqueeze(0)>0.5
        probFile1 = os.path.relpath(maskFile, os.path.join(root, 'slice/test')).replace('npy', 'pt')
        probFile1 = os.path.join(root, 'oProb/unetpp0D/s_test', probFile1)
        y_pred1 = torch.load(probFile1,weights_only=False).unsqueeze(0).unsqueeze(0)>0.5
        y = torch.from_numpy(np.load(maskFile)).unsqueeze(0).unsqueeze(0).cuda(0)
        d0= compute_dice(y_pred0, y).item()
        d1= compute_dice(y_pred1, y).item()
        dScore = pd.concat([dScore, pd.DataFrame
                            ([[os.path.relpath(maskFile,os.path.join(root,'slice/s_test')), d0, d1]], 
                                columns=['maskTfile', 'd0','d1'])])
    
    # probIFiles = glob.glob(f'/workspaces/data/brain_meningioma/oProb/unet1s/s_test/pt/BraTS-MEN-RT-{id}-2_slice_*.pt')

AttributeError: module 'os' has no attribute 'relpath'

In [38]:
maskTfile

'npy/BraTS-MEN-RT-0109-1_slice_*_mask.npy'

In [41]:
dScore['dadd'] = dScore['d1'] - dScore['d0']

In [42]:
dScore.sort_values(by='dadd', ascending=False).head()

,maskTfile,d0,d1,dadd
0,npy/BraTS-MEN-RT-0264-1_slice_*_mask.npy,0.000000,0.800000,0.800000
0,npy/BraTS-MEN-RT-0053-1_slice_*_mask.npy,0.000000,0.742857,0.742857
0,npy/BraTS-MEN-RT-0165-1_slice_*_mask.npy,0.000000,0.742358,0.742358
0,npy/BraTS-MEN-RT-0264-1_slice_*_mask.npy,0.102041,0.772727,0.670686
0,npy/BraTS-MEN-RT-0165-1_slice_*_mask.npy,0.190045,0.853755,0.663710
